In [38]:
import pandas as pd

In [39]:
obsidian_flames = pd.read_csv('Obsidian_Flames.csv', index_col = 'Unnamed: 0')
obsidian_flames

,Product,Foil,Date Range,Market Price
0,Absol ex - 135/197,['Holofoil Market Price: '],7/18 to 7/20,$0.00
1,Absol ex - 135/197,['Holofoil Market Price: '],7/21 to 7/23,$0.00
2,Absol ex - 135/197,['Holofoil Market Price: '],7/24 to 7/26,$0.00
3,Absol ex - 135/197,['Holofoil Market Price: '],7/27 to 7/29,$0.00
4,Absol ex - 135/197,['Holofoil Market Price: '],7/30 to 8/1,$4.87
...,...,...,...,...
12925,Zigzagoon,['Reverse Holofoil Market Price: '],10/7 to 10/9,$0.06
12926,Zigzagoon,[' Market Price: '],10/10 to 10/12,$0.03
12927,Zigzagoon,['Reverse Holofoil Market Price: '],10/10 to 10/12,$0.06
12928,Zigzagoon,[' Market Price: '],10/13 to 10/15,$0.03


In [40]:
obsidian_flames['Foil'].unique()

array(["['Holofoil Market Price: ']", "[' Market Price: ']",
       "['Reverse Holofoil Market Price: ']"], dtype=object)

In [41]:
obsidian_flames.replace({'Foil' : { "[' Market Price: ']" : "Normal",
                               "['Reverse Holofoil Market Price: ']" : "Reverse Holofoil", 
                               "['Holofoil Market Price: ']" : "Holofoil"}}, inplace=True)
obsidian_flames

,Product,Foil,Date Range,Market Price
0,Absol ex - 135/197,Holofoil,7/18 to 7/20,$0.00
1,Absol ex - 135/197,Holofoil,7/21 to 7/23,$0.00
2,Absol ex - 135/197,Holofoil,7/24 to 7/26,$0.00
3,Absol ex - 135/197,Holofoil,7/27 to 7/29,$0.00
4,Absol ex - 135/197,Holofoil,7/30 to 8/1,$4.87
...,...,...,...,...
12925,Zigzagoon,Reverse Holofoil,10/7 to 10/9,$0.06
12926,Zigzagoon,Normal,10/10 to 10/12,$0.03
12927,Zigzagoon,Reverse Holofoil,10/10 to 10/12,$0.06
12928,Zigzagoon,Normal,10/13 to 10/15,$0.03


In [42]:
sum(obsidian_flames['Market Price'] == "$0.00") / len(obsidian_flames['Market Price'] == "$0.00")
# Just 20% of the dataset has $0.00 market price

0.20471771075019335

In [43]:
of_cleaned = obsidian_flames[obsidian_flames['Market Price'] != '$0.00']
of_cleaned

,Product,Foil,Date Range,Market Price
4,Absol ex - 135/197,Holofoil,7/30 to 8/1,$4.87
5,Absol ex - 135/197,Holofoil,8/2 to 8/4,$4.74
6,Absol ex - 135/197,Holofoil,8/5 to 8/7,$4.74
7,Absol ex - 135/197,Holofoil,8/8 to 8/10,$2.04
8,Absol ex - 135/197,Holofoil,8/11 to 8/13,$0.80
...,...,...,...,...
12925,Zigzagoon,Reverse Holofoil,10/7 to 10/9,$0.06
12926,Zigzagoon,Normal,10/10 to 10/12,$0.03
12927,Zigzagoon,Reverse Holofoil,10/10 to 10/12,$0.06
12928,Zigzagoon,Normal,10/13 to 10/15,$0.03


In [44]:
of_cleaned = of_cleaned.sort_values(by=['Product', 'Foil'])
of_cleaned

,Product,Foil,Date Range,Market Price
4,Absol ex - 135/197,Holofoil,7/30 to 8/1,$4.87
5,Absol ex - 135/197,Holofoil,8/2 to 8/4,$4.74
6,Absol ex - 135/197,Holofoil,8/5 to 8/7,$4.74
7,Absol ex - 135/197,Holofoil,8/8 to 8/10,$2.04
8,Absol ex - 135/197,Holofoil,8/11 to 8/13,$0.80
...,...,...,...,...
12921,Zigzagoon,Reverse Holofoil,10/1 to 10/3,$0.06
12923,Zigzagoon,Reverse Holofoil,10/4 to 10/6,$0.06
12925,Zigzagoon,Reverse Holofoil,10/7 to 10/9,$0.06
12927,Zigzagoon,Reverse Holofoil,10/10 to 10/12,$0.06


In [45]:
of_cleaned['Market Price'] = of_cleaned['Market Price'].replace({'\$':''}, regex = True)
of_cleaned

,Product,Foil,Date Range,Market Price
4,Absol ex - 135/197,Holofoil,7/30 to 8/1,4.87
5,Absol ex - 135/197,Holofoil,8/2 to 8/4,4.74
6,Absol ex - 135/197,Holofoil,8/5 to 8/7,4.74
7,Absol ex - 135/197,Holofoil,8/8 to 8/10,2.04
8,Absol ex - 135/197,Holofoil,8/11 to 8/13,0.80
...,...,...,...,...
12921,Zigzagoon,Reverse Holofoil,10/1 to 10/3,0.06
12923,Zigzagoon,Reverse Holofoil,10/4 to 10/6,0.06
12925,Zigzagoon,Reverse Holofoil,10/7 to 10/9,0.06
12927,Zigzagoon,Reverse Holofoil,10/10 to 10/12,0.06


In [46]:
of_cleaned = of_cleaned.reset_index(drop=True)
of_cleaned

,Product,Foil,Date Range,Market Price
0,Absol ex - 135/197,Holofoil,7/30 to 8/1,4.87
1,Absol ex - 135/197,Holofoil,8/2 to 8/4,4.74
2,Absol ex - 135/197,Holofoil,8/5 to 8/7,4.74
3,Absol ex - 135/197,Holofoil,8/8 to 8/10,2.04
4,Absol ex - 135/197,Holofoil,8/11 to 8/13,0.80
...,...,...,...,...
10278,Zigzagoon,Reverse Holofoil,10/1 to 10/3,0.06
10279,Zigzagoon,Reverse Holofoil,10/4 to 10/6,0.06
10280,Zigzagoon,Reverse Holofoil,10/7 to 10/9,0.06
10281,Zigzagoon,Reverse Holofoil,10/10 to 10/12,0.06


In [47]:
# tells us whether the product is the same as the previous line product
match_prev_product = [of_cleaned['Product'][i] == of_cleaned['Product'][i+1] for i in range(len(of_cleaned['Product'])-1)]
match_prev_foil = [of_cleaned['Foil'][i] == of_cleaned['Foil'][i+1] for i in range(len(of_cleaned['Foil'])-1)]
prev_market_price = of_cleaned['Market Price'][:-1]
match_prev_product and match_prev_foil

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 T

In [48]:
prev_market_price

0        4.87
1        4.74
2        4.74
3        2.04
4        0.80
         ... 
10277    0.06
10278    0.06
10279    0.06
10280    0.06
10281    0.06
Name: Market Price, Length: 10282, dtype: object

In [49]:
of_cleaned = of_cleaned[1:]
of_cleaned = of_cleaned.reset_index(drop=True)
of_cleaned['Previous Market Price'] = prev_market_price
of_cleaned

,Product,Foil,Date Range,Market Price,Previous Market Price
0,Absol ex - 135/197,Holofoil,8/2 to 8/4,4.74,4.87
1,Absol ex - 135/197,Holofoil,8/5 to 8/7,4.74,4.74
2,Absol ex - 135/197,Holofoil,8/8 to 8/10,2.04,4.74
3,Absol ex - 135/197,Holofoil,8/11 to 8/13,0.80,2.04
4,Absol ex - 135/197,Holofoil,8/14 to 8/16,0.86,0.80
...,...,...,...,...,...
10277,Zigzagoon,Reverse Holofoil,10/1 to 10/3,0.06,0.06
10278,Zigzagoon,Reverse Holofoil,10/4 to 10/6,0.06,0.06
10279,Zigzagoon,Reverse Holofoil,10/7 to 10/9,0.06,0.06
10280,Zigzagoon,Reverse Holofoil,10/10 to 10/12,0.06,0.06


In [50]:
prev_match = match_prev_product and match_prev_foil

In [53]:
of_cleaned = of_cleaned[prev_match]
of_cleaned = of_cleaned.reset_index(drop=True)
of_cleaned

,Product,Foil,Date Range,Market Price,Previous Market Price
0,Absol ex - 135/197,Holofoil,8/2 to 8/4,4.74,4.87
1,Absol ex - 135/197,Holofoil,8/5 to 8/7,4.74,4.74
2,Absol ex - 135/197,Holofoil,8/8 to 8/10,2.04,4.74
3,Absol ex - 135/197,Holofoil,8/11 to 8/13,0.80,2.04
4,Absol ex - 135/197,Holofoil,8/14 to 8/16,0.86,0.80
...,...,...,...,...,...
9892,Zigzagoon,Reverse Holofoil,10/1 to 10/3,0.06,0.06
9893,Zigzagoon,Reverse Holofoil,10/4 to 10/6,0.06,0.06
9894,Zigzagoon,Reverse Holofoil,10/7 to 10/9,0.06,0.06
9895,Zigzagoon,Reverse Holofoil,10/10 to 10/12,0.06,0.06
